In [1]:
import pandas as pd

df1 = pd.read_csv("./optimalalphabeta/dearr_cont/dearr0.1_0.06.csv")
#df1 = pd.read_csv("bordascores_top3.csv")
df2 = pd.read_csv("df_mfeatures.csv")
df3 = pd.read_csv("BinnedMetaFeatures.csv")
df1 = df1[['File','FeatureAlgo']]

In [2]:
merged_df = df1.merge(df2, on='File').merge(df3, on='File')
merged_df.to_csv("KnowledgeBase.csv", index=False)

In [3]:
df1 = pd.read_csv("majority_voted_labels_clusteredearr.csv")
merged_df = df1.merge(df2, on='File').merge(df3, on='File')
merged_df.to_csv("KnowledgeBase_majorityvotecluster.csv", index=False)

# Run from here

In [13]:
df1 = pd.read_csv("KnowledgeBase.csv")
df2 = pd.read_csv("KnowledgeBase_majorityvotecluster.csv")

In [14]:
df_char_raw1 = df1[[ 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]

df_quality_raw1 = df1[['Completeness', 'Conciseness', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'LabelIssuesPerc', 'FeatureAlgo']]

df_raw1 = df1[['Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]


df_char_bin1 = df1[['cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins','FeatureAlgo']]

df_quality_bin1 = df1[['Completeness_bins',
       'Conciseness_bins', 'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins','FeatureAlgo']]

df_bin1 = df1[['Completeness_bins',
       'Conciseness_bins', 'cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins', 'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins', 'FeatureAlgo']]

In [15]:
df_char_raw2 = df2[[ 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]

df_quality_raw2 = df2[['Completeness', 'Conciseness', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'LabelIssuesPerc', 'FeatureAlgo']]

df_raw2 = df2[['Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]


df_char_bin2 = df2[['cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins','FeatureAlgo']]

df_quality_bin2 = df2[['Completeness_bins',
       'Conciseness_bins', 'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins','FeatureAlgo']]

df_bin2 = df2[['Completeness_bins',
       'Conciseness_bins', 'cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins', 'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins', 'FeatureAlgo']]

## RAW DATA

# kNN

In [10]:
import numpy as np 

from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


In [19]:
#k-NN
def evaluate_knn(df, random_state=42):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets1 = {
    'df_raw': df_raw1,
    'df_char_raw': df_char_raw1,
    'df_quality_raw': df_quality_raw1
}

results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_knn(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************KNN - 0.1_0.06**************\n")
print(comparison_df1)

datasets2 = {
    'df_raw': df_raw2,
    'df_char_raw': df_char_raw2,
    'df_quality_raw': df_quality_raw2
}

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_knn(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************KNN - Majority clustering**************\n")
print(comparison_df2)

**************KNN - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.506878                    0.493122   
df_char_raw                     0.506878                    0.493122   
df_quality_raw                  0.499471                    0.500529   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.571429        0.428571  0.516104  0.571429  0.542356   
df_char_raw         0.571429        0.428571  0.516104  0.571429  0.542356   
df_quality_raw           0.4             0.6  0.331815       0.4  0.361196   

                                                 Confusion Matrix  
df_raw          [[7, 0, 2, 1, 0], [2, 0, 0, 0, 0], [2, 2, 13, ...  
df_char_raw     [[7, 0, 2, 1, 0], [2, 0, 0, 0, 0], [2, 2, 13, ...  
df_quality_raw  [[2, 0, 5, 3, 0], [0, 0, 2, 0, 0], [6, 0, 12, ...  


**************KNN - Majority clustering**************

               Cross-Vali

## Decision trees

In [20]:
#decision

def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }


results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_decision_tree(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************DT - 0.1_0.06**************\n")
print(comparison_df1)

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_decision_tree(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************DT - Majority clustering**************\n")
print(comparison_df2)

**************DT - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.623016                    0.376984   
df_char_raw                     0.594444                    0.405556   
df_quality_raw                  0.536772                    0.463228   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.542857        0.457143  0.540977  0.542857  0.539683   
df_char_raw         0.514286        0.485714  0.494874  0.514286  0.501253   
df_quality_raw      0.371429        0.628571  0.390765  0.371429  0.378947   

                                                 Confusion Matrix  
df_raw          [[5, 0, 2, 1, 0], [1, 0, 1, 0, 0], [3, 0, 13, ...  
df_char_raw     [[5, 0, 2, 1, 0], [1, 0, 1, 0, 0], [3, 1, 13, ...  
df_quality_raw  [[2, 1, 4, 0, 1], [2, 0, 0, 0, 0], [6, 0, 11, ...  


**************DT - Majority clustering**************

               Cross-Valida

## Distance based

In [21]:
results1 = {}
results2 = {}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results1[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }


for name, df in datasets2.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results2[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }



results_df1 = pd.DataFrame(results1).transpose()
results_df2 = pd.DataFrame(results2).transpose()

print("**************Distance based - 0.1_0.06**************\n")
print(results_df1)

print("\n\n**************Distance based - Majority clustering**************\n")
print(results_df2)


**************Distance based - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.530924                    0.469076   
df_char_raw                     0.530924                    0.469076   
df_quality_raw                  0.444202                    0.555798   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.485714        0.514286   0.44517  0.485714  0.463957   
df_char_raw         0.485714        0.514286   0.44517  0.485714  0.463957   
df_quality_raw      0.428571        0.571429  0.424047  0.428571  0.417724   

                                                 Confusion Matrix  
df_raw          [[4, 0, 4, 0], [1, 0, 1, 0], [3, 0, 13, 4], [2...  
df_char_raw     [[4, 0, 4, 0], [1, 0, 1, 0], [3, 0, 13, 4], [2...  
df_quality_raw  [[5, 0, 2, 0, 1], [1, 0, 1, 0, 0], [6, 0, 10, ...  


**************Distance based - Majority clustering**************

   

/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d191

## Unification

In [22]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=42):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)

    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


results1 = {}
results2 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_unification(df)

results_df1 = pd.DataFrame(results1).transpose()

for name, df in datasets2.items():
    results2[name] = evaluate_unification(df)

results_df2 = pd.DataFrame(results2).transpose()

print("**************Unification - 0.1_0.06**************\n")
print(results_df1)

print("\n\n**************Unification - Majority clustering**************\n")
print(results_df2)


/home/d19125691/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


**************Unification - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.722222                    0.277778   
df_char_raw                     0.722222                    0.277778   
df_quality_raw                  0.387937                    0.612063   

               Test Accuracy Test Error Rate Precision Recall  F1 Score  \
df_raw                   NaN             NaN         0      0         0   
df_char_raw              NaN             NaN         0      0         0   
df_quality_raw           0.5             0.5  0.833333    0.5  0.611111   

                                                 Confusion Matrix  
df_raw                                                         []  
df_char_raw                                                    []  
df_quality_raw  [[1, 0, 0, 1], [1, 2, 1, 0], [0, 0, 0, 0], [0,...  


**************Unification - Majority clustering**************

               Cross-

/home/d19125691/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## BIN data

## kNN

In [30]:
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

results1 = {}

datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}
datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_raw': df_quality_bin2
}

for name, df in datasets1.items():
    results1[name] = evaluate_knn(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************KNN - 0.1_0.06**************\n")
print(comparison_df1)


results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_knn(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************KNN - Majority clustering**************\n")
print(comparison_df2)

**************KNN - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.413228                    0.586772   
df_char_bin                     0.522222                    0.477778   
df_quality_bin                  0.434921                    0.565079   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.342857        0.657143  0.316902  0.342857  0.328389   
df_char_bin         0.485714        0.514286  0.407937  0.485714  0.443392   
df_quality_bin      0.371429        0.628571  0.334485  0.371429  0.350158   

                                                 Confusion Matrix  
df_bin          [[1, 0, 7, 0], [2, 0, 0, 0], [8, 0, 11, 1], [2...  
df_char_bin     [[2, 0, 6, 0], [1, 0, 0, 1], [4, 0, 15, 1], [2...  
df_quality_bin  [[2, 0, 6, 0], [1, 0, 1, 0], [8, 0, 11, 1], [2...  


**************KNN - Majority clustering**************

               Cross-Vali

## Decision tree

In [31]:
def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
        
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

results1 = {}
datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}
datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_raw': df_quality_bin2
}
for name, df in datasets1.items():
    results1[name] = evaluate_decision_tree(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************DT - 0.1_0.06**************\n")
print(comparison_df1)

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_decision_tree(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************DT - Majority clustering**************\n")
print(comparison_df2)

**************DT - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.478307                    0.521693   
df_char_bin                     0.478571                    0.521429   
df_quality_bin                  0.384392                    0.615608   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.371429        0.628571  0.352381  0.371429      0.36   
df_char_bin         0.428571        0.571429    0.3953  0.428571   0.41049   
df_quality_bin      0.314286        0.685714  0.273292  0.314286  0.292359   

                                                 Confusion Matrix  
df_bin          [[2, 0, 6, 0, 0], [1, 0, 0, 1, 0], [7, 0, 11, ...  
df_char_bin     [[2, 0, 6, 0, 0], [1, 0, 0, 1, 0], [5, 0, 13, ...  
df_quality_bin  [[0, 0, 8, 0], [2, 0, 0, 0], [9, 0, 11, 0], [1...  


**************DT - Majority clustering**************

               Cross-Valida

## Distance based

In [26]:
datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}
datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_bin': df_quality_bin2
}
results1 = {}
results2 = {}


metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
        
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results1[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df1 = pd.DataFrame(results1).transpose()
print("***************Distance based - 0.1_0.06*************")
print(results_df1)



for name, df in datasets2.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
        
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results2[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df2 = pd.DataFrame(results2).transpose()
print("\n\n***************Distance based - Majority cluster*************")

print(results_df2)

***************Distance based - 0.1_0.06*************
               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.387059                    0.612941   
df_char_bin                     0.474118                    0.525882   
df_quality_bin                  0.323361                    0.676639   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.314286        0.685714  0.304762  0.314286  0.309222   
df_char_bin         0.371429        0.628571  0.370748  0.371429  0.370112   
df_quality_bin      0.371429        0.628571  0.275132  0.371429  0.316109   

                                                 Confusion Matrix  
df_bin          [[1, 0, 7, 0, 0], [0, 0, 0, 2, 0], [6, 0, 10, ...  
df_char_bin     [[0, 0, 8, 0, 0], [1, 0, 0, 1, 0], [7, 0, 11, ...  
df_quality_bin  [[0, 0, 8, 0], [0, 0, 1, 1], [3, 0, 13, 4], [0...  


***************Distance based - Majority cluster*************
        

/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d191

## Unification

In [27]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=42):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}
datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_bin': df_quality_bin2
}
results1 = {}
results2 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_unification(df)
for name, df in datasets2.items():
    results2[name] = evaluate_unification(df)

results_df1 = pd.DataFrame(results1).transpose()
results_df2 = pd.DataFrame(results2).transpose()

print("***************Unification - 0.1_0.06**********************")
print(results_df1)
print("***************Unification - majority voting**********************")
print(results_df2)


***************Unification - 0.1_0.06**********************
               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.468067                    0.531933   
df_char_bin                     0.315363                    0.684637   
df_quality_bin                  0.248192                    0.751808   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.368421        0.631579  0.424812  0.368421  0.389516   
df_char_bin         0.242424        0.757576  0.470041  0.242424   0.31527   
df_quality_bin       0.30303         0.69697  0.587205   0.30303  0.390374   

                                                 Confusion Matrix  
df_bin          [[3, 0, 2, 1], [0, 0, 1, 0], [4, 1, 4, 2], [0,...  
df_char_bin     [[1, 0, 2, 4, 0], [0, 0, 1, 0, 0], [7, 3, 7, 3...  
df_quality_bin  [[2, 1, 2, 1, 2], [1, 0, 0, 0, 0], [4, 2, 8, 4...  
***************Unification - majority voting**********************

## Standard Deviation

In [32]:
import pandas as pd
df_normalised1 = pd.read_csv("NormalisedDataset_01006.csv")
df_normalised2 = pd.read_csv("NormalisedDataset_majoritycluster.csv")

df_normalised_quality1 = df_normalised1[['Completeness_normalized', 'Conciseness_normalized',  'ClassImbRatio_normalized', 'ClassOverlapPerc_normalized',
       'OutlierPerc_normalized',  'LabelIssuesPerc_normalized', 'FeatureAlgo_normalized']]
df_normalised_char1 = df_normalised1[[ 'cor.mean_normalized', 'cov.mean_normalized',
       'eigenvalues.mean_normalized', 'g_mean.mean_normalized',
       'h_mean.mean_normalized', 'iq_range.mean_normalized',
       'kurtosis.mean_normalized', 'mad.mean_normalized',
       'max.mean_normalized', 'mean.mean_normalized', 'median.mean_normalized',
       'min.mean_normalized', 'nr_cor_attr_normalized', 'nr_norm_normalized',
       'nr_outliers_normalized', 'range.mean_normalized', 'sd.mean_normalized',
       'skewness.mean_normalized', 'sparsity.mean_normalized',
       't_mean.mean_normalized', 'var.mean_normalized',
        'attr_to_inst_normalized',
       'inst_to_attr_normalized', 'nr_attr_normalized', 'nr_bin_normalized',
       'nr_inst_normalized', 'nr_num_normalized', 'attr_conc.mean_normalized',
       'attr_ent.mean_normalized', 'nUnique_normalized', 'ena_normalized', 'snr.mean_normalized',
       'cEntropy_normalized', 'FeatureAlgo_normalized']]
df_normalised_quality2 = df_normalised2[['Completeness_normalized', 'Conciseness_normalized',  'ClassImbRatio_normalized', 'ClassOverlapPerc_normalized',
       'OutlierPerc_normalized',  'LabelIssuesPerc_normalized', 'FeatureAlgo_normalized']]
df_normalised_char2 = df_normalised2[[ 'cor.mean_normalized', 'cov.mean_normalized',
       'eigenvalues.mean_normalized', 'g_mean.mean_normalized',
       'h_mean.mean_normalized', 'iq_range.mean_normalized',
       'kurtosis.mean_normalized', 'mad.mean_normalized',
       'max.mean_normalized', 'mean.mean_normalized', 'median.mean_normalized',
       'min.mean_normalized', 'nr_cor_attr_normalized', 'nr_norm_normalized',
       'nr_outliers_normalized', 'range.mean_normalized', 'sd.mean_normalized',
       'skewness.mean_normalized', 'sparsity.mean_normalized',
       't_mean.mean_normalized', 'var.mean_normalized',
        'attr_to_inst_normalized',
       'inst_to_attr_normalized', 'nr_attr_normalized', 'nr_bin_normalized',
       'nr_inst_normalized', 'nr_num_normalized', 'attr_conc.mean_normalized',
       'attr_ent.mean_normalized', 'nUnique_normalized', 'ena_normalized', 'snr.mean_normalized',
       'cEntropy_normalized', 'FeatureAlgo_normalized']]

## kNN

In [34]:
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets1 = {
    'df_normalised': df_normalised1,
    'df_normalised_char': df_normalised_char1,
    'df_normalised_quality': df_normalised_quality1
}

results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_knn(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************KNN - 0.1_0.06**************\n")
print(comparison_df1)

datasets2 = {
    'df_normalised': df_normalised2,
    'df_normalised_char': df_normalised_char2,
    'df_normalised_quality': df_normalised_quality2
}

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_knn(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************KNN - Majority clustering**************\n")
print(comparison_df2)

**************KNN - 0.1_0.06**************

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.449735                    0.550265   
df_normalised_char                     0.470899                    0.529101   
df_normalised_quality                  0.428042                    0.571958   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.542857        0.457143  0.487912  0.542857   
df_normalised_char         0.428571        0.571429  0.388889  0.428571   
df_normalised_quality      0.457143        0.542857  0.380952  0.457143   

                       F1 Score  \
df_normalised          0.508844   
df_normalised_char      0.39599   
df_normalised_quality  0.413605   

                                                        Confusion Matrix  
df_normalised          [[5, 0, 3, 0], [1, 0, 0, 1], [5, 0, 14, 1], [2...  
df_normalised_char     [[5, 0, 3, 0], [0, 0, 2,

## Decision tree

In [35]:
def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }


results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_decision_tree(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************DT - 0.1_0.06**************\n")
print(comparison_df1)

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_decision_tree(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************DT - Majority clustering**************\n")
print(comparison_df2)

**************DT - 0.1_0.06**************

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.623545                    0.376455   
df_normalised_char                     0.485714                    0.514286   
df_normalised_quality                  0.492593                    0.507407   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.514286        0.485714  0.515934  0.514286   
df_normalised_char         0.457143        0.542857  0.431232  0.457143   
df_normalised_quality      0.485714        0.514286  0.391461  0.485714   

                       F1 Score  \
df_normalised          0.501587   
df_normalised_char     0.438723   
df_normalised_quality  0.414736   

                                                        Confusion Matrix  
df_normalised          [[7, 0, 1, 0, 0], [0, 0, 1, 1, 0], [4, 0, 11, ...  
df_normalised_char     [[4, 0, 4, 0], [2, 0, 0, 

## Distance based

In [36]:
datasets1 = {
    'df_normalised': df_normalised1,
    'df_normalised_char': df_normalised_char1,
    'df_normalised_quality': df_normalised_quality1
}

results1 = {}
results2 = {}

datasets2 = {
    'df_normalised': df_normalised2,
    'df_normalised_char': df_normalised_char2,
    'df_normalised_quality': df_normalised_quality2
}


metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results1[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }

for name, df in datasets2.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results2[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }


results_df1 = pd.DataFrame(results1).transpose()

print(results_df1)


results_df2 = pd.DataFrame(results2).transpose()

print(results_df2)

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.456639                    0.543361   
df_normalised_char                     0.497143                    0.502857   
df_normalised_quality                   0.39916                     0.60084   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.542857        0.457143  0.474258  0.542857   
df_normalised_char         0.457143        0.542857  0.428571  0.457143   
df_normalised_quality      0.428571        0.571429  0.390476  0.428571   

                       F1 Score  \
df_normalised          0.506234   
df_normalised_char         0.44   
df_normalised_quality  0.401732   

                                                        Confusion Matrix  
df_normalised          [[4, 0, 4, 0], [1, 0, 0, 1], [3, 0, 15, 2], [1...  
df_normalised_char     [[3, 0, 4, 0, 1], [2, 0, 0, 0, 0], [5, 0, 13, ...  
df_normalised_qu

/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d191

## Unification

In [37]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


results1 = {}
results2 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_unification(df)
for name, df in datasets2.items():
    results2[name] = evaluate_unification(df)

results_df1 = pd.DataFrame(results1).transpose()
results_df2 = pd.DataFrame(results2).transpose()

print("***************Unification - 0.1_0.06**********************")
print(results_df1)
print("***************Unification - majority voting**********************")
print(results_df2)


***************Unification - 0.1_0.06**********************
                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.431263                    0.568737   
df_normalised_char                     0.319669                    0.680331   
df_normalised_quality                  0.253957                    0.746043   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised                  0.25            0.75  0.295833      0.25   
df_normalised_char         0.333333        0.666667  0.447619  0.333333   
df_normalised_quality       0.30303         0.69697  0.581818   0.30303   

                       F1 Score  \
df_normalised          0.265873   
df_normalised_char     0.381944   
df_normalised_quality  0.387205   

                                                        Confusion Matrix  
df_normalised          [[1, 0, 2, 0], [1, 0, 0, 0], [2, 0, 2, 3], [0,...  
df_normalised_char     [[3, 0, 